# PySparkのTips

細かいTips、テクニックをまとめる。

In [1]:
from glob import glob

import polars as pl
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.functions import vector_to_array
import numpy as np

# Create a SparkSession。pythonからsparkを使う場合、セッションの作成が必要。
spark = SparkSession.builder.appName("Testing PySpark Example").getOrCreate()

# デフォルトのログレベルだと大量にログが出力されるので限定する。
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/03 14:52:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/03 14:52:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
# 各データ読み込み
df_receipt = spark.read.parquet("../../../100knocks-preprocess/docker/work/data/receipt.parquet")

# 店舗データ
df_store = spark.read.parquet("../../../100knocks-preprocess/docker/work/data/store.parquet")

# 顧客データ
df_customer = spark.read.parquet("../../../100knocks-preprocess/docker/work/data/customer.parquet")

# 製品データ
df_product = spark.read.parquet("../../../100knocks-preprocess/docker/work/data/product.parquet")

# 製品データ
df_category = spark.read.parquet("../../../100knocks-preprocess/docker/work/data/category.parquet")

## 特定カラムのユニークな値をリストとしてすべて取得
下記のような方法がある。どちらにせよめんどくさい。

In [8]:
# collectの結果の各値は対象カラムをキーとする辞書のような形で取得できる
[v["gender_cd"] for v in df_customer.select("gender_cd").distinct().collect()]

['0', '9', '1']

In [4]:
df_customer.select("gender_cd").dropDuplicates().rdd.flatMap(lambda x: x).collect()

['0', '9', '1']

In [7]:
# Dataframeとして取得したいなら.distinctでOK
df_customer.select("gender_cd").distinct().show()

+---------+
|gender_cd|
+---------+
|        0|
|        9|
|        1|
+---------+

